In [ ]:
# Fourier Transform

# Step 1: Import libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# Step 2: Load published Google Sheet as CSV
sheet_url = "https://docs.google.com/spreadsheets/d/e/2PACX-1vTnBHi20fVOhu9sJhfk8-6Sxh_M0WmrIfncUunLQrUEKcd9fRzX24as2yoWR8Q-YzNz_SZXcnzK99Xi/pub?output=csv"
df = pd.read_csv(sheet_url)
df.columns = ['timestamp', 'rain_value']

# Step 3: Clean and convert data
df.dropna(inplace=True)
df['timestamp'] = pd.to_numeric(df['timestamp'], errors='coerce')
df['rain_value'] = pd.to_numeric(df['rain_value'], errors='coerce')
df.dropna(inplace=True)

# Step 4: Sort and compute sampling info
df = df.sort_values('timestamp')
timestamps = df['timestamp'].values
values = df['rain_value'].values
interval = np.mean(np.diff(timestamps))
fs = 1 / interval  # sampling frequency

print(f"Sampling Interval: {interval:.2f}s | Sampling Frequency: {fs:.4f} Hz")

# Step 5: Remove DC offset
signal = values - np.mean(values)

# Step 6: FFT
N = len(signal)
fft_result = np.fft.fft(signal)
frequencies = np.fft.fftfreq(N, d=interval)

# Step 7: Plot frequency spectrum
plt.figure(figsize=(10,5))
plt.plot(frequencies[:N//2], np.abs(fft_result[:N//2]))
plt.title("Frequency Spectrum (FFT) of Rain Sensor Signal")
plt.xlabel("Frequency (Hz)")
plt.ylabel("Magnitude")
plt.grid(True)
plt.show()

# Step 8: Low-pass filtering (remove high-frequency noise)
cutoff_hz = 0.1  # change this as needed based on signal
filtered_fft = fft_result.copy()
filtered_fft[np.abs(frequencies) > cutoff_hz] = 0

# Step 9: Reconstruct signal using inverse FFT
filtered_signal = np.fft.ifft(filtered_fft).real

# Step 10: Plot original vs filtered signal
plt.figure(figsize=(10,5))
plt.plot(df['timestamp'], values, label='Original Signal')
plt.plot(df['timestamp'], filtered_signal, label='Filtered Signal (Low-pass)', linestyle='--')
plt.title("Noise Reduction via Frequency Filtering")
plt.xlabel("Time")
plt.ylabel("Rain Sensor Value")
plt.legend()
plt.grid(True)
plt.show()

In [ ]:

#Fourier Series

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# Load published CSV from Google Sheets
sheet_url = "https://docs.google.com/spreadsheets/d/e/2PACX-1vTnBHi20fVOhu9sJhfk8-6Sxh_M0WmrIfncUunLQrUEKcd9fRzX24as2yoWR8Q-YzNz_SZXcnzK99Xi/pub?output=csv"
df = pd.read_csv(sheet_url)
df.columns = ['timestamp', 'rain_value']

# Clean and prepare
df.dropna(inplace=True)
df['timestamp'] = pd.to_numeric(df['timestamp'], errors='coerce')
df['rain_value'] = pd.to_numeric(df['rain_value'], errors='coerce')
df.dropna(inplace=True)

# Sort and convert time
df = df.sort_values('timestamp')
t = df['timestamp'].values
y = df['rain_value'].values
t = t - t[0]  # Normalize time to start at 0
T = t[-1]  # total time span
N = len(t)

# Fourier Series parameters
n_harmonics = 10
w0 = 2 * np.pi / T  # fundamental angular frequency

# Compute a0
a0 = (2 / T) * np.trapz(y, t)

# Compute an, bn
an = []
bn = []
for n in range(1, n_harmonics + 1):
    cos_term = np.cos(n * w0 * t)
    sin_term = np.sin(n * w0 * t)
    an_n = (2 / T) * np.trapz(y * cos_term, t)
    bn_n = (2 / T) * np.trapz(y * sin_term, t)
    an.append(an_n)
    bn.append(bn_n)

# Reconstruct signal
y_reconstructed = a0 / 2 * np.ones_like(t)
for n in range(1, n_harmonics + 1):
    y_reconstructed += an[n-1] * np.cos(n * w0 * t) + bn[n-1] * np.sin(n * w0 * t)

# Plot original vs reconstructed
plt.figure(figsize=(10, 5))
plt.plot(t, y, label="Original Signal")
plt.plot(t, y_reconstructed, label=f"Fourier Series ({n_harmonics} harmonics)", linestyle='--')
plt.title("Rain Sensor Data - Fourier Series Approximation")
plt.xlabel("Time (s)")
plt.ylabel("Sensor Value")
plt.legend()
plt.grid(True)
plt.show()

In [ ]:

# Decision Tree Algorithm

# Install required libraries
!pip install -q scikit-learn pandas

# Import libraries
import pandas as pd
import numpy as np
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix

# Load rain sensor data from Google Sheets
sheet_url = "https://docs.google.com/spreadsheets/d/e/2PACX-1vTnBHi20fVOhu9sJhfk8-6Sxh_M0WmrIfncUunLQrUEKcd9fRzX24as2yoWR8Q-YzNz_SZXcnzK99Xi/pub?output=csv"
df = pd.read_csv(sheet_url)
df.columns = ['timestamp', 'rain_value']
df.dropna(inplace=True)
df['rain_value'] = pd.to_numeric(df['rain_value'], errors='coerce')
df.dropna(inplace=True)

# Classify rain levels
def classify_rain(value):
    if value >= 3000:
        return "No Rain"
    elif value >= 2000:
        return "Light"
    elif value >= 1000:
        return "Moderate"
    else:
        return "Heavy"

df['rain_class'] = df['rain_value'].apply(classify_rain)

# Create feature set from last 10 rain readings
N = 10
X = []
y = []

for i in range(len(df) - N):
    window = df['rain_value'].iloc[i:i+N].values
    label = df['rain_class'].iloc[i+N]
    X.append(window)
    y.append(label)

X = np.array(X)
y = np.array(y)

# Split (30% test, no shuffle to preserve order)
X_train, X_test, y_train, y_test = train_test_split(X[:-1], y[:-1], test_size=0.3, shuffle=False)

# Train Decision Tree Classifier
clf = DecisionTreeClassifier(max_depth=4, random_state=42)
clf.fit(X_train, y_train)

# Predict next classification (using latest 10 values)
latest_window = X[-1].reshape(1, -1)
predicted_class = clf.predict(latest_window)[0]

print(f" Predicted classification for the next reading: {predicted_class}")

# Evaluate performance on test set
y_pred = clf.predict(X_test)

print("\n Classification Report (Test Set):")
print(classification_report(y_test, y_pred))

print("Confusion Matrix:")
print(confusion_matrix(y_test, y_pred))

# Show Actual vs Predicted for inspection
print("\n Actual vs Predicted:")
for actual, pred in zip(y_test, y_pred):
    print(f"Actual: {actual:<10} | Predicted: {pred}")

In [ ]:

# Random Forest Algorithm

# Install required libraries
!pip install -q scikit-learn pandas

# Import libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from collections import Counter
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix

# Load rain sensor data from Google Sheets
sheet_url = "https://docs.google.com/spreadsheets/d/e/2PACX-1vTnBHi20fVOhu9sJhfk8-6Sxh_M0WmrIfncUunLQrUEKcd9fRzX24as2yoWR8Q-YzNz_SZXcnzK99Xi/pub?output=csv"
df = pd.read_csv(sheet_url)
df.columns = ['timestamp', 'rain_value']
df.dropna(inplace=True)
df['rain_value'] = pd.to_numeric(df['rain_value'], errors='coerce')
df.dropna(inplace=True)

# Classify rain levels
def classify_rain(value):
    if value >= 3000:
        return "No Rain"
    elif value >= 2000:
        return "Light"
    elif value >= 1000:
        return "Moderate"
    else:
        return "Heavy"

df['rain_class'] = df['rain_value'].apply(classify_rain)

# Create feature set using last 10 values
N = 10
X = []
y = []

for i in range(len(df) - N):
    window = df['rain_value'].iloc[i:i+N].values
    label = df['rain_class'].iloc[i+N]
    X.append(window)
    y.append(label)

X = np.array(X)
y = np.array(y)

# Stratified split with preserved class ratios
X_train, X_test, y_train, y_test = train_test_split(
    X[:-1], y[:-1],
    test_size=0.4,
    stratify=y[:-1],     # Maintain class balance
    random_state=42
)

# Check class distribution
print("Class distribution in training set:", Counter(y_train))
print("Class distribution in test set:", Counter(y_test))

# Train Random Forest Classifier
clf = RandomForestClassifier(n_estimators=100, max_depth=8, random_state=42)
clf.fit(X_train, y_train)

# Predict next classification using latest 10 readings
latest_window = X[-1].reshape(1, -1)
predicted_class = clf.predict(latest_window)[0]

print(f"Predicted classification for the next reading: {predicted_class}")

# Evaluate model
y_pred = clf.predict(X_test)

print("\n Classification Report (Test Set):")
print(classification_report(y_test, y_pred))

print("Confusion Matrix:")
print(confusion_matrix(y_test, y_pred))

# Actual vs Predicted output
print("\n Actual vs Predicted:")
for actual, pred in zip(y_test, y_pred):
    print(f"Actual: {actual:<10} | Predicted: {pred}")